Student: Arora, Sanjana, V00966221

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as sps
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.metrics import confusion_matrix, roc_auc_score

In [ ]:
df = pd.read_csv('elections_clean.csv')

In [ ]:
subset = df[['Education', 'Religion', 'EthnicMale','EthnicFemale']].copy()

In [ ]:
df_transformed = pd.get_dummies(subset)
len(df_transformed.columns)

17

In [ ]:
df_transformed.head()

,Education_BachelorOrHigher,Education_College,Education_LessthanDiploma,Education_OnlyDiploma,Religion_Amish,Religion_Catholic,Religion_Christian Generic,Religion_Mormon,Religion_Other,Religion_Other Misc,EthnicMale_ASIAN_MALE_rate,EthnicMale_BLACK_MALE_rate,EthnicMale_NATIVE_AMERICAN_MALE_rate,EthnicMale_WHITE_MALE_rate,EthnicFemale_BLACK_FEMALE_rate,EthnicFemale_NATIVE_AMERICAN_FEMALE_rate,EthnicFemale_WHITE_FEMALE_rate
0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1
1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1
2,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
3,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1
4,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1


In [ ]:
X = df_transformed.iloc[:,:].values
Y = df['Democrat'].values.reshape(-1,1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, shuffle = True)

In [ ]:
dataset = df[['Education', 'Religion', 'EthnicMale', 'EthnicFemale', 'Democrat']].copy()

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=50, max_depth=3, random_state=42)

In [ ]:
rf_clf.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=3, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
Y_pred = rf_clf.predict(X_train)

In [ ]:
print("The training mean absolute error: {0:.4f}\n".format(mean_absolute_error(Y_train,Y_pred))) 
print("The training accuracy score: {0:.4f}\n".format(accuracy_score(Y_train,Y_pred))) 

The training mean absolute error: 0.0977

The training accuracy score: 0.9023



In [ ]:
Y_predtest = rf_clf.predict(X_test)

In [ ]:
print("The testing mean absolute error: {0:.4f}\n".format(mean_absolute_error(Y_test,Y_predtest))) 
print("The testing accuracy score: {0:.4f}\n".format(accuracy_score(Y_test,Y_predtest)))

The testing mean absolute error: 0.0784

The testing accuracy score: 0.9216



*Conclusion relating to random forests in the end*

**Bonus Question**: *Random Forest from Scratch*

The codes for random forest have been inherited from the codes used for preparing decision trees as the core logics for creating the decision trees following the entropy based split criteria remain the same.

I append all the created trees in a dictionary in the "**RandomForestTrain**" function. 
Use the majority voting of the predictions output by each of the tree in "**RandomForest_Predict**" function.The default value used for prediction is 1. 

Store all the predicted values in a dataframe columns and calculate the mean squared error and accuracy scores in "**RandomForestTest**" function. 

Further, there is a change in decisionclassifier function, wherein, subspace is getting sampled by randomly choosing sqrt(total number of features) for each subspace.

In [ ]:
#Calculating the entropy

def entropy(Y): # Y is the Target Columns
  values, counts = np.unique(Y,return_counts = True)
  for i in range(len(values)):
    prob = counts[i]/np.sum(counts) # calculating the probability
    h = np.sum(-(prob)*np.log2(prob))
  return h

In [ ]:
#Calculating the Information Gain
def InfoGain(dataset,split_feature_name, Y):
  total_h = entropy(dataset[Y]) 
  values, counts = np.unique(dataset[split_feature_name],return_counts = True)
  #initiating the weighted entropy from 0
  weighted_h = 0
  for i in range(len(values)):
    prob = counts[i]/np.sum(counts)
    feature = dataset.where(dataset[split_feature_name]==values[i]).dropna()[Y]
    h = entropy(feature)
    # Adding values to the entropy as we traverse through the features
    weighted_h = weighted_h + prob*h
  IG = total_h - weighted_h
  return IG

In [ ]:
def DecisionClassifier(dataset,originaldata,features,Y,depth=20,filter_node_class = None):
      
      # Checking if all target values have same values left
      if (len(np.unique(dataset[Y])) <= 1):
          
          return np.unique(dataset[Y])[0] # return the value and length of the tree
      
      # if dataset has got empty
      elif (len(dataset)==0) :
        
        # index of the mode target feature value in dataset
        index = np.argmax(np.unique(originaldata[Y],return_counts=True)[1])
        mode = np.unique(originaldata[Y])[index]
        return mode #returning the target feature value occuring maximum number of times in original dataset
      
      # if the number of features have got zero, returning the mode target feature value indicated by the parent node that called the DecisionClassifier recursion
      elif (len(features) ==0):
        return filter_node_class
      
      
      # if none of the conditions are true, growing the tree
      else:
        
        # growing the tree
        # assigning parent node class feature values as the selected target feature value having max. no. of occurrences
          index = np.argmax(np.unique(dataset[Y],return_counts=True)[1])
          filter_node_class  = np.unique(dataset[Y])[index]
          
          #Implementing the subspace sampling. Draw a number of m = sqrt(p) feature
          features = np.random.choice(features,size=np.int(np.sqrt(len(features))),replace=False)
         # selecting the feature which best splits the dataset 
          item_vals = [InfoGain(dataset,feature,Y) for feature in features]
          
          # best feature is the feature having maximum information gain about the target feature
          best_featureindex = np.argmax(item_vals)
          best_feature = features[best_featureindex]
         
          # tree structure
          tree = {best_feature:{}}
          
          # iterating over all features except for the best feature
          features = [i for i in features if i != best_feature]
        
          #finding the dataset corresponding the best feature for further analysis
          for vals in np.unique(dataset[best_feature]):
              
              sub_data = dataset.where(dataset[best_feature] == vals).dropna()  
              subtree = DecisionClassifier(sub_data,dataset,features,Y,depth,filter_node_class)
            
            
            # adding subtree, under the grown tree
              tree[best_feature][vals] = subtree
          return tree

In [ ]:
def predict(query,tree,default = 1):
        
    for key in list(query.keys()):
        if key in list(tree.keys()):
            try:
                result = tree[key][query[key]] 
            except:
                return default
            result = tree[key][query[key]]
            if isinstance(result,dict):
                return predict(query,result)

            else:
                return result

In [ ]:
def train_validation_split(dataset):
    validation_split = .3
    dataset_size = len(dataset)
    split = int(np.floor(validation_split * dataset_size))
    training_data = dataset.iloc[split:].reset_index(drop=True)
    validation_data = dataset.iloc[:split].reset_index(drop=True)
    return training_data,validation_data

In [ ]:
def RandomForestTrain(dataset,Y,number_of_Trees):
    #Creating an empty list to store the number of trees
    random_forest_sub_tree = []
    
    # To create the number of trees as mentioned by user
    for i in range(number_of_Trees):
        # Shuffling the dataset to create a number of bootstrap sampled datasets from the original dataset 
        bootstrap_data = dataset.sample(frac=1,replace=True)
        
        #Create a training and a testing dataset by calling the train_validation_split function
        bootstrap_training_data, bootstrap_testing_data = train_validation_split(bootstrap_data)
        
        
        #Growing the tree
        #Creating trees with bootstrapped datasets
        random_forest_sub_tree.append(DecisionClassifier(bootstrap_training_data,bootstrap_training_data,bootstrap_training_data.drop(labels=[Y],axis=1).columns,Y))
        
    return random_forest_sub_tree

In [ ]:
def RandomForestTest(data,random_forest_tree,target_attribute_name):
  #Creating an empty column to store prediction values
    data['predictions'] = None
    for i in range(len(data)):
        queries = data.iloc[i,:].drop(target_attribute_name).to_dict()
        #Storing the prediction outcomes in the column
        data.loc[i,'predictions'] = RandomForest_Predict(queries,random_forest_tree,default=1)
    #calculating the accuracy scores
    accuracy = sum(data['predictions'] == data[target_attribute_name])/len(data)*100
    # Calculating the mean squared error
    MAE = np.mean(np.square(data['predictions'] - np.array(data[target_attribute_name].astype(int))))
    return accuracy, MAE

In [ ]:
def RandomForest_Predict(query,random_forest_tree,default=1):
    predictions = []
    #for each tree in the randomforest dictionary
    for tree in random_forest_tree:
        predictions.append(predict(query,tree,default))
        # taking majority voting for decision
    return sps.mode(predictions)[0][0]

In [ ]:
training_data1,testing_data = train_validation_split(dataset)
random_forest = RandomForest_Train(training_data1,'Democrat',50)

In [ ]:
Testing_accuracy, Testing_MSE = RandomForestTest(testing_data,random_forest,'Democrat')
print('The testing accuracy of model made from scratch:',Testing_accuracy )
print('The testing MAE of model:',Testing_MSE )

The testing accuracy of model made from scratch: 87.0625662778367
The testing MAE of model: 0.12937433722163308


In [ ]:
Training_accuracy, Training_MSE = RandomForestTest(training_data1,random_forest,'Democrat')
print('The training accuracy of made from scratch:',Training_accuracy )
print('The training MAE of model:',Training_MSE )

The training accuracy of made from scratch: 87.92007266121708
The training MAE of model: 0.12079927338782924


Referred online materials from https://stats.stackexchange.com/questions/295868/why-is-tree-correlation-a-problem-when-working-with-bagging/295883

2. https://www.analyticsvidhya.com/blog/2016/04/tree-based-algorithms-complete-tutorial-scratch-in-python/

According to me, the random forests are not performing better on this data due to the following observations:

 1. The mean absolute training error of random forest  0.0909 is higher than mean absolute training error of decision trees 0.0854157201272149 made by using the sklearn library. 

2. Also, **the decision trees created manually**  are performing **better** as compared to the random forests. The testing Mean Absolute Error (MAE) for random forest **0.0911** is higher than the MAE of manually created decision trees **0.08695652173913043**.

3. While the differences in mean absolute testing error of random forest (0.9089) and mean absolute testing error of decision tree (0.09427966101694915) are quite low.

4. Training MAE of manually created Decision Trees (0.08764759309718438) is much better than training MAE of random forest 0.0909.

The probable reason of random forest not performing better than decision trees could be pruning the decision trees to the depth of 3.Further, the random forest performs bootstrapping on the input training data, it is possible that there are multiple repeating feature values that keep on occuring in shuffling of datasets and features.